# Processos de Poisson

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [66]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [67]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [68]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo introduz o [Poisson process](https://en.wikipedia.org/wiki/Poisson_point_process), que é um modelo utilizado para descrever eventos que ocorrem a intervalos aleatórios.

Como exemplo de um processo Poisson, vamos modelar a marcação de golos no futebol, que é inglês americano para o jogo a que todos os outros chamam "futebol".

Utilizaremos os golos marcados num jogo para estimar o parâmetro de um processo Poisson; depois utilizaremos a distribuição posterior para fazer previsões.

E vamos resolver o problema do Campeonato do Mundo.

## O problema do Campeonato do Mundo

Na final do Campeonato Mundial de Futebol de 2018, a França derrotou a Croácia por 4 golos a 2. Com base neste resultado:

1. Até que ponto devemos estar confiantes de que a França é a melhor equipa?

2. Se as mesmas equipas jogassem de novo, qual seria a hipótese de a França voltar a ganhar?

Para responder a estas questões, temos de tomar algumas decisões de modelização.

* Primeiro, vou assumir que para qualquer equipa contra outra equipa existe uma taxa de pontuação desconhecida, medida em golos por jogo, que denotarei com a variável Python `lam` ou a letra grega $\lambda$, pronunciada "lambda".

* Segundo, vou assumir que um golo é igualmente provável durante qualquer minuto de um jogo.  Assim, num jogo de 90 minutos, a probabilidade de marcar um golo durante qualquer minuto é de $\lambda/90$.

* Em terceiro lugar, vou assumir que uma equipa nunca marca duas vezes durante o mesmo minuto.

Claro que nenhuma destas suposições é completamente verdadeira no mundo real, mas penso que são simplificações razoáveis.

Como disse George Box: "Todos os modelos estão errados; alguns são úteis".

(https://en.wikipedia.org/wiki/All_models_are_wrong).

Neste caso, o modelo é útil porque se estes pressupostos são 

É verdade, pelo menos aproximadamente, que o número de golos marcados num jogo segue uma distribuição Poisson, pelo menos aproximadamente.

## A Distribuição Poisson

Se o número de golos marcados num jogo segue um [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) com uma taxa de marcação de golos, $\lambda$, a probabilidade de marcar golos $k$ é

$$$lambda^k {\i1}exp(-lambda) ~/~ k!$$

por qualquer valor não-negativo de $k$.

SciPy fornece um objecto `poisson` que representa uma distribuição Poisson.

Podemos criar um com $\lambda=1,4$ como este:

In [69]:
from scipy.stats import poisson

lam = 1.4
dist = poisson(lam)
type(dist)

O resultado é um objecto que representa uma variável aleatória "congelada" e fornece `pmf`, que avalia a função de massa probabilística da distribuição de Poisson.

In [70]:
k = 4
dist.pmf(k)

Este resultado implica que se a taxa média de marcação de golos for de 1,4 golos por jogo, a probabilidade de marcar 4 golos num jogo é de cerca de 4%.

Vamos utilizar a seguinte função para fazer um `Pmf` que representa uma distribuição Poisson.

In [71]:
from empiricaldist import Pmf

def make_poisson_pmf(lam, qs):
    """Make a Pmf of a Poisson distribution."""
    ps = poisson(lam).pmf(qs)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

O `make_poisson_pmf` toma como parâmetros a taxa de pontuação, `lam`, e um conjunto de quantidades, `qs`, onde deve avaliar o PMF de Poisson.  Devolve um objecto `Pmf`.

Por exemplo, aqui está a distribuição dos golos marcados para `lam=1,4`, calculados para valores de `k` de 0 a 9.

In [72]:
import numpy as np

lam = 1.4
goals = np.arange(10)
pmf_goals = make_poisson_pmf(lam, goals)

E aqui está o que parece.

In [73]:
from utils import decorate

def decorate_goals(title=''):
    decorate(xlabel='Number of goals',
        ylabel='PMF',
        title=title)

In [74]:
pmf_goals.bar(label=r'Poisson distribution with $\lambda=1.4$')

decorate_goals('Distribution of goals scored')

Os resultados mais prováveis são 0, 1, e 2; valores mais altos são possíveis mas cada vez mais improváveis.

Os valores acima de 7 são insignificantes.

Esta distribuição mostra que se conhecermos a taxa de marcação de golos, podemos prever o número de golos.

Agora vamos inverter a situação: tendo em conta uma série de objectivos, o que podemos dizer sobre a taxa de marcação de objectivos?

Para responder a isto, precisamos de pensar na distribuição prévia do `lam', que representa a gama de valores possíveis e as suas probabilidades antes de vermos a pontuação.

## A Distribuição Gama

Se alguma vez viu um jogo de futebol, tem alguma informação sobre o `lam`.  Na maioria dos jogos, as equipas marcam alguns golos cada uma.  Em casos raros, uma equipa pode marcar mais de 5 golos, mas quase nunca marcam mais de 10.

Utilizando [data from previous World Cups](https://www.statista.com/statistics/269031/goals-scored-per-game-at-the-fifa-world-cup-since-1930/), estimo que cada equipa marca cerca de 1,4 golos por jogo, em média.  Assim, vou definir a média de 1,4 golos.

Para uma boa equipa contra uma má, esperamos que `lam` seja mais alta; para uma má equipa contra uma boa, esperamos que seja mais baixa.

Para modelar a distribuição das taxas de marcação de objectivos, vou usar um [gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution), que escolhi porque:

1. A taxa de pontuação dos golos é contínua e não negativa, e a distribuição gama é apropriada para este tipo de quantidade.

2. A distribuição gama tem apenas um parâmetro, "alfa", que é a média.  Portanto, é fácil construir uma distribuição gama com a média que desejamos.

3. Como veremos, a forma da distribuição gama é uma escolha razoável, tendo em conta o que sabemos sobre futebol.

E há mais uma razão, que vou revelar em <<_ConjugatePriors>>>>.

SciPy fornece `gamma`, que cria um objecto que representa uma distribuição gama.

E o objecto `gamma` fornece o `pdf`, que avalia a ** função de densidade de probabilidade** (PDF) da distribuição gama.

Eis como o utilizamos.

In [75]:
from scipy.stats import gamma

alpha = 1.4
qs = np.linspace(0, 10, 101)
ps = gamma(alpha).pdf(qs)

O parâmetro, "alfa", é o meio da distribuição.

Os `qs` são valores possíveis de `lam` entre 0 e 10.

Os `ps' são ** densidades de probabilidade***, que podemos pensar como probabilidades não normalizadas.

Para normalizá-los, podemos colocá-los num "Pmf" e chamar-lhes "normalizar":

In [76]:
from empiricaldist import Pmf

prior = Pmf(ps, qs)
prior.normalize()

O resultado é uma discreta aproximação de uma distribuição gama.

Aqui está o que parece.

In [77]:
def decorate_rate(title=''):
    decorate(xlabel='Goal scoring rate (lam)',
        ylabel='PMF',
        title=title)

In [78]:
prior.plot(style='--', label='prior', color='C5')
decorate_rate(r'Prior distribution of $\lambda$')

Esta distribuição representa o nosso conhecimento prévio sobre a marcação de golos: O "Lâm" é normalmente inferior a 2, ocasionalmente tão alto como 6, e raramente mais alto que isso.  

E podemos confirmar que a média é de cerca de 1,4.

In [79]:
prior.mean()

Como habitualmente, pessoas razoáveis poderiam discordar sobre os detalhes do anterior, mas isto é suficientemente bom para começar.  Vamos fazer uma actualização.

## A Actualização

Suponha que lhe é dada a taxa de marcação de golos, $\lambda$, e lhe é pedido que calcule a probabilidade de marcar um certo número de golos, $k$.  Esta é precisamente a pergunta a que respondemos através do cálculo do PMF de Poisson.

Por exemplo, se $\lambda$ for 1,4, a probabilidade de marcar 4 golos num jogo é de 1,4:

In [80]:
lam = 1.4
k = 4
poisson(lam).pmf(4)

Agora suponha que temos um conjunto de valores possíveis por $\lambda$; podemos calcular a probabilidade dos dados para cada valor hipotético de `lam`, como este:

In [81]:
lams = prior.qs
k = 4
likelihood = poisson(lams).pmf(k)

E isso é tudo o que precisamos para fazer a actualização.

Para obter a distribuição posterior, multiplicamos o anterior pelas probabilidades que acabamos de calcular e normalizamos o resultado.

A função que se segue encapsula estas etapas.

In [82]:
def update_poisson(pmf, data):
    """Update Pmf with a Poisson likelihood."""
    k = data
    lams = pmf.qs
    likelihood = poisson(lams).pmf(k)
    pmf *= likelihood
    pmf.normalize()

O primeiro parâmetro é o anterior; o segundo é o número de objectivos.

No exemplo, a França marcou 4 golos, por isso vou fazer uma cópia do anterior e actualizá-lo com os dados.

In [83]:
france = prior.copy()
update_poisson(france, 4)

Eis como se parece a distribuição posterior, juntamente com a anterior.

In [84]:
prior.plot(style='--', label='prior', color='C5')
france.plot(label='France posterior', color='C3')

decorate_rate('Posterior distribution for France')

Os dados, `k=4`, fazem-nos pensar que os valores mais altos de `lam` são mais prováveis e os valores mais baixos são menos prováveis.  Assim, a distribuição posterior é deslocada para a direita.

Vamos fazer o mesmo pela Croácia:

In [85]:
croatia = prior.copy()
update_poisson(croatia, 2)

E aqui estão os resultados.

In [86]:
prior.plot(style='--', label='prior', color='C5')
croatia.plot(label='Croatia posterior', color='C0')

decorate_rate('Posterior distribution for Croatia')

Aqui estão os meios posteriores para estas distribuições.

In [87]:
print(croatia.mean(), france.mean())

A média da distribuição prévia é de cerca de 1,4.

Após a Croácia marcar 2 golos, a sua média posterior é de 1,7, que está perto do ponto médio do anterior e dos dados.

Da mesma forma, após a França marcar 4 golos, a sua média posterior é de 2,7.

Estes resultados são típicos de uma actualização Bayesiana: a localização da distribuição posterior é um compromisso entre o anterior e os dados.

## Probabilidade de Superioridade

Agora que temos uma distribuição posterior para cada equipa, podemos responder à primeira pergunta: Até que ponto devemos estar confiantes de que a França é a melhor equipa?

No modelo, "melhor" significa ter uma taxa de pontuação mais elevada contra o adversário.  Podemos utilizar as distribuições posteriores para calcular a probabilidade de que um valor aleatório retirado da distribuição da França exceda um valor retirado da Croácia.

Uma maneira de o fazer é enumerar todos os pares de valores das duas distribuições, somando a probabilidade total de um valor exceder o outro.

In [88]:
def prob_gt(pmf1, pmf2):
    """Compute the probability of superiority."""
    total = 0
    for q1, p1 in pmf1.items():
        for q2, p2 in pmf2.items():
            if q1 > q2:
                total += p1 * p2
    return total

Isto é semelhante ao método que utilizamos em <<_Addends>> para calcular a distribuição de uma soma.

Eis como o utilizamos:

In [89]:
prob_gt(france, croatia)

O `Pmf` fornece uma função que faz a mesma coisa.

In [90]:
Pmf.prob_gt(france, croatia)

Os resultados são ligeiramente diferentes porque `Pmf.prob_gt` utiliza operadores de matriz em vez de loops `para` loops.

Seja como for, o resultado está próximo dos 75%.  Assim, com base num jogo, temos uma confiança moderada de que a França é de facto a melhor equipa.

É claro que devemos lembrar que este resultado se baseia no pressuposto de que a taxa de pontuação é constante.

Na realidade, se uma equipa estiver a perder por um golo, poderá jogar de forma mais agressiva no final do jogo, tornando-os mais propensos a marcar, mas também mais propensos a desistir de um golo adicional.

Como sempre, os resultados são apenas tão bons como o modelo.

## Prevendo a desforra

Agora podemos responder à segunda pergunta: Se as mesmas equipas jogassem novamente, qual seria a hipótese de a Croácia ganhar?

Para responder a esta pergunta, vamos gerar a "distribuição preditiva posterior", que é o número de golos que esperamos que uma equipa marque.

Se soubéssemos a taxa de marcação de golos, `lam', a distribuição de golos seria uma distribuição de Poisson com o parâmetro `lam'.

Uma vez que não conhecemos o `lam', a distribuição de objectivos é uma mistura de uma distribuição de Poisson com diferentes valores de `lam'.

Primeiro vou gerar uma sequência de objectos `Pmf`, uma para cada valor de `lam`.

In [91]:
pmf_seq = [make_poisson_pmf(lam, goals) 
           for lam in prior.qs]

A figura seguinte mostra como são estas distribuições para alguns valores de `lam`.

In [92]:
import matplotlib.pyplot as plt

for i, index in enumerate([10, 20, 30, 40]):
    plt.subplot(2, 2, i+1)
    lam = prior.qs[index]
    pmf = pmf_seq[index]
    pmf.bar(label=f'$\lambda$ = {lam}', color='C3')
    decorate_goals()

A distribuição preditiva é uma mistura destes objectos `Pmf`, ponderados com as probabilidades posteriores.

Podemos utilizar `make_mixture` de <<_GeneralMixtures>> para calcular esta mistura.

In [93]:
from utils import make_mixture

pred_france = make_mixture(france, pmf_seq)

Eis a distribuição preditiva do número de golos que a França marcaria numa desforra.

In [94]:
pred_france.bar(color='C3', label='France')
decorate_goals('Posterior predictive distribution')

Esta distribuição representa duas fontes de incerteza: não sabemos o valor real do `lam`, e mesmo que soubéssemos, não saberíamos o número de golos no jogo seguinte.

Eis a distribuição preditiva para a Croácia.

In [95]:
pred_croatia = make_mixture(croatia, pmf_seq)

In [96]:
pred_croatia.bar(color='C0', label='Croatia')
decorate_goals('Posterior predictive distribution')

Podemos utilizar estas distribuições para calcular a probabilidade de a França ganhar, perder, ou amarrar a desforra.

In [97]:
win = Pmf.prob_gt(pred_france, pred_croatia)
win

In [98]:
lose = Pmf.prob_lt(pred_france, pred_croatia)
lose

In [99]:
tie = Pmf.prob_eq(pred_france, pred_croatia)
tie

Assumindo que a França ganha metade dos empates, a sua hipótese de ganhar a desforra é de cerca de 65%.

In [100]:
win + tie/2

Isto é um pouco menor do que a sua probabilidade de superioridade, que é 75%. E isso faz sentido, porque estamos menos seguros sobre o resultado de um único jogo do que sobre as taxas de marcação de golos.

Mesmo que a França seja a melhor equipa, eles podem perder o jogo.

## A Distribuição Exponencial

Como exercício no final deste caderno, terá a oportunidade de trabalhar na seguinte variação sobre o problema do Campeonato do Mundo:

>No Campeonato do Mundo de Futebol de 2014, a Alemanha jogou contra o Brasil numa partida semifinal. A Alemanha marcou ao fim de 11 minutos e novamente aos 23 minutos. Nesse momento do jogo, quantos golos esperaria que a Alemanha fizesse ao fim de 90 minutos? Qual era a probabilidade de marcarem mais 5 golos (como, de facto, fizeram)?

Nesta versão, note-se que os dados não são o número de objectivos num período de tempo fixo, mas o tempo entre objectivos.

Para calcular a probabilidade de dados como este, podemos aproveitar novamente a teoria dos processos de Poisson.  Se cada equipa tiver uma taxa constante de pontuação de objectivos, esperamos que o tempo entre objectivos siga um [exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution).

Se a taxa de pontuação dos objectivos for $\lambda$, a probabilidade de ver um intervalo entre objectivos de $t$ é proporcional ao PDF da distribuição exponencial:

$$lambda \exp(-lambda t)$$

Como $t$ é uma quantidade contínua, o valor desta expressão não é uma probabilidade; é uma densidade de probabilidade.  No entanto, é proporcional à probabilidade dos dados, pelo que podemos utilizá-la como uma probabilidade numa actualização Bayesiana.

SciPy fornece `expon', que cria um objecto que representa uma distribuição exponencial.

No entanto, não toma o `lam` como um parâmetro da forma que se poderia esperar, o que torna estranho trabalhar com ele.

Uma vez que o PDF da distribuição exponencial é tão fácil de avaliar, vou usar a minha própria função.

In [101]:
def expo_pdf(t, lam):
    """Compute the PDF of the exponential distribution."""
    return lam * np.exp(-lam * t)

Para ver como é a distribuição exponencial, vamos assumir novamente que `lam` é 1,4; podemos calcular a distribuição de $t$ desta forma:

In [102]:
lam = 1.4
qs = np.linspace(0, 4, 101)
ps = expo_pdf(qs, lam)
pmf_time = Pmf(ps, qs)
pmf_time.normalize()

E aqui está o que parece:

In [103]:
def decorate_time(title=''):
    decorate(xlabel='Time between goals (games)',
             ylabel='PMF',
             title=title)

In [104]:
pmf_time.plot(label='exponential with $\lambda$ = 1.4')

decorate_time('Distribution of time between goals')

É contra-intuitivo, mas é verdade, que o momento mais provável para marcar um golo é imediatamente.  Depois disso, a probabilidade de cada intervalo sucessivo é um pouco menor.

Com uma taxa de pontuação de 1,4, é possível que uma equipa leve mais do que um jogo para marcar um golo, mas é improvável que leve mais do que dois jogos.

## Resumo

Este capítulo introduz três novas distribuições, pelo que pode ser difícil mantê-las direitas.

Vamos rever:

* Se um sistema satisfaz os pressupostos de um modelo de Poisson, o número de eventos num período de tempo segue uma distribuição de Poisson, que é uma distribuição discreta com quantidades inteiras de 0 a infinito. Na prática, podemos normalmente ignorar quantidades de baixa probabilidade acima de um limite finito.

* Também sob o modelo Poisson, o intervalo entre eventos segue uma distribuição exponencial, que é uma distribuição contínua com quantidades de 0 a infinito. Por ser contínua, é descrita por uma função de densidade de probabilidade (PDF) em vez de uma função de massa de probabilidade (PMF). Mas quando utilizamos uma distribuição exponencial para calcular a probabilidade dos dados, podemos tratar as densidades como probabilidades não normalizadas.

* A Poisson e as distribuições exponenciais são parametrizadas por uma taxa de evento, designada por $\lambda$ ou `lam`.

* Para a distribuição prévia de $\lambda$, utilizei uma distribuição gama, que é uma distribuição contínua com quantidades de 0 a infinito, mas aproximei-a com um PMF discreto e limitado. A distribuição gama tem um parâmetro, denotado $\alpha$ ou `alpha`, que é também a sua média.

Escolhi a distribuição gama porque a forma é consistente com o nosso conhecimento de base sobre as taxas de marcação de golos.

Há outras distribuições que poderíamos ter utilizado; contudo, veremos em <<_ConjugatePriors>> que a distribuição gama pode ser uma escolha particularmente boa.

Mas temos algumas coisas a fazer antes de lá chegarmos, começando com estes exercícios.

## Exercícios

**Exercício:** Vamos terminar o exercício que começámos:

>No Campeonato do Mundo de Futebol de 2014, a Alemanha jogou contra o Brasil numa partida semifinal. A Alemanha marcou ao fim de 11 minutos e novamente aos 23 minutos. Nesse momento do jogo, quantos golos esperaria que a Alemanha fizesse ao fim de 90 minutos? Qual era a probabilidade de marcarem mais 5 golos (como, de facto, fizeram)?

Aqui estão os passos que recomendo:

1. Começando com a mesma gama antes utilizada no problema anterior, calcular a probabilidade de marcar um golo após 11 minutos para cada valor possível de `lam`.  Não se esqueça de converter todas as vezes em jogos em vez de minutos.

2. Calcular a distribuição posterior de `lam` para a Alemanha após o primeiro objectivo.

3. Calcule a probabilidade de marcar outro golo após mais 12 minutos e faça outra actualização.  Plotar o anterior, o posterior após um golo, e o posterior após dois golos.

4. Calcular a distribuição preditiva posterior dos golos que a Alemanha poderá marcar durante o tempo restante do jogo, `90-23` minutos.  Nota: Terá de pensar em como gerar objectivos previstos para uma fracção de um jogo.

5. Calcular a probabilidade de marcar 5 ou mais golos durante o tempo restante.

In [105]:
# Solution goes here

In [106]:
# Solution goes here

In [107]:
# Solution goes here

In [108]:
# Solution goes here

In [109]:
# Solution goes here

In [110]:
# Solution goes here

In [111]:
# Solution goes here

In [112]:
# Solution goes here

In [113]:
# Solution goes here

**Exercício:** Voltando à primeira versão do problema do Campeonato do Mundo.  Suponhamos que a França e a Croácia jogam uma desforra.  Qual é a probabilidade de a França marcar primeiro?

Dica: Calcule a distribuição preditiva posterior até ao primeiro objectivo, fazendo uma mistura de distribuições exponenciais.  Pode utilizar a seguinte função para fazer um DPP que se aproxime de uma distribuição exponencial.

In [114]:
def make_expo_pmf(lam, high):
    """Make a PMF of an exponential distribution.
    
    lam: event rate
    high: upper bound on the interval `t`
    
    returns: Pmf of the interval between events
    """
    qs = np.linspace(0, high, 101)
    ps = expo_pdf(qs, lam)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

In [115]:
# Solution goes here

In [116]:
# Solution goes here

In [117]:
# Solution goes here

In [118]:
# Solution goes here

**Exercício:** Nas finais da Liga Nacional de Hóquei 2010-11 (NHL), minha amada Boston

Bruins jogou uma série de campeonatos de melhor de sete contra os desprezados

Vancouver Canucks.  Boston perdeu os dois primeiros jogos 0-1 e 2-3, depois

ganhou os dois jogos seguintes 8-1 e 4-0.  Neste ponto da série, o que

é a probabilidade de Boston ganhar o próximo jogo, e o que é

a sua probabilidade de ganhar o campeonato?

Para escolher uma distribuição prévia, recebi algumas estatísticas de

http://www.nhl.com, especificamente os objectivos médios por jogo

para cada equipa na época de 2010-11.  A distribuição é bem modelada por uma distribuição gama com média de 2,8.

De que forma pensa que o resultado destes jogos pode violar os pressupostos do modelo Poisson?  Como é que estas violações afectariam as suas previsões?

In [119]:
# Solution goes here

In [120]:
# Solution goes here

In [121]:
# Solution goes here

In [122]:
# Solution goes here

In [123]:
# Solution goes here

In [124]:
# Solution goes here

In [125]:
# Solution goes here

In [126]:
# Solution goes here

In [127]:
# Solution goes here

In [128]:
# Solution goes here

In [129]:
# Solution goes here

In [130]:
# Solution goes here